In [ ]:
!pip install openai # Instala a biblioteca OpenAI, que permite interagir com a API da OpenAI para usar modelos de linguagem, como o GPT-3.
!pip install langchain # Instala a biblioteca LangChain, que facilita o desenvolvimento de aplicativos de linguagem com modelos de linguagem.
!pip install langchain-openai # Instala o componente LangChain específico para integração com a OpenAI API.
!pip install langchain-community # Instala a comunidade de componentes LangChain que fornecem funcionalidades extras.
!pip install qdrant-client # Instala a biblioteca cliente Qdrant, que permite se comunicar com um servidor Qdrant, um banco de dados vetorial para armazenar e consultar dados vetoriais.
!pip install pypdf2 # Instala a biblioteca PyPDF, que permite a leitura e manipulação de arquivos PDF.

In [ ]:
!pip install transformers sentence-transformers
!pip install torch


In [39]:
from pathlib import Path
from transformers import RagTokenizer, RagTokenForGeneration
from sentence_transformers import SentenceTransformer
from langchain.docstore.document import Document
from langchain_community.vectorstores import Qdrant
from PyPDF2 import PdfReader  # Certifique-se de ter PyPDF2 instalado

class SentenceTransformerWrapper:
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, documents):
        # Extrai o conteúdo de cada documento e gera embeddings
        texts = [doc.content for doc in documents]  # Alterado de page_content para content
        return self.model.encode(texts).astype('float32')

# Função para carregar PDFs e gerar embeddings
def carregar_pdfs_para_qdrant(diretorio_pdfs: str):
    documentos = []

    # Itera sobre todos os PDFs no diretório
    for pdf_path in Path(diretorio_pdfs).glob("*.pdf"):
        with open(pdf_path, "rb") as file:
            reader = PdfReader(file)
            texto = ""
            for page in reader.pages:
                texto += page.extract_text()  # Extrai o texto de cada página

            # Converte o texto do PDF em um objeto `Document` corretamente
            documentos.append(Document(page_content=texto))  # Usando 'content' em vez de 'page_content'

    # Inicializa o wrapper do SentenceTransformer
    embedding_model = SentenceTransformerWrapper('sentence-transformers/all-MiniLM-L6-v2')

    # Instanciando Qdrant em memória
    qdrant = Qdrant.from_documents(
        documents=documentos,
        embedding=embedding_model,
        location=":memory:",  # Banco de dados temporário em memória
        collection_name="chatbot"
    )
    return qdrant

# Carregar PDFs (garanta que o diretório tenha PDFs)
qdrant = carregar_pdfs_para_qdrant("./meus_pdfs")

# Configurar o RAG
tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-base")
model = RagTokenForGeneration.from_pretrained("facebook/rag-token-base")

# Função para criar prompt personalizado usando RAG
def custom_prompt(query: str):
    results = qdrant.similarity_search(query, k=3)
    source_knowledge = "\n".join([x.content for x in results])  # Alterado para content

    augment_prompt = f"""Use o contexto abaixo para responder à pergunta.

    Contexto:
    {source_knowledge}

    Pergunta: {query}"""

    return augment_prompt

# Limite de histórico de mensagens
def limitar_mensagens(messages, max_messages=5):
    if len(messages) > max_messages:
        messages = messages[-max_messages:]  # Mantém apenas as últimas mensagens
    return messages

# Função para gerar resposta usando RAG
def gerar_resposta(query):
    prompt = custom_prompt(query)
    inputs = tokenizer([prompt], return_tensors="pt", padding=True, truncation=True)
    outputs = model.generate(inputs['input_ids'], num_return_sequences=1)
    resposta = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return resposta[0]

# Testando o fluxo com uma query
query = str(input("Usuário: "))
messages = [query]
messages = limitar_mensagens(messages, max_messages=5)

resposta = gerar_resposta(query)
print("RAG_FCI:", resposta)

AttributeError: 'str' object has no attribute 'content'

In [34]:
import openai
from langchain_openai import ChatOpenAI
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)
from langchain_community.vectorstores import Qdrant
from langchain.docstore.document import Document
from langchain_openai import OpenAIEmbeddings
from PyPDF2 import PdfReader  # Biblioteca para leitura de PDFs

# Configuração do modelo OpenAI
chave_api = "YOUR_API_KEY"

#Instância da classe `ChatOpenAI` que será usada para interagir com o modelo de chat.
chat = ChatOpenAI(
    model='gpt-3.5-turbo',
    openai_api_key=chave_api
)
'''
# Definindo persistência no banco de dados vetorial
db_location = "./qdrant_db"  # Definindo um caminho de disco para persistência
qdrant = Qdrant(
    location=db_location,
    collection_name="chatbot"
)
'''
# Função para carregar PDFs e gerar embeddings
def carregar_pdfs_para_qdrant(diretorio_pdfs: str):
    from pathlib import Path
    documentos = []

    # Itera sobre todos os PDFs no diretório
    for pdf_path in Path(diretorio_pdfs).glob("*.pdf"):
        reader = PdfReader(str(pdf_path))
        texto = ""
        for page in reader.pages:
            texto += page.extract_text()  # Extrai o texto de cada página
        #Converte o texto do PDF em um objeto `Document`
        documentos.append(Document(page_content=texto))

    #import os
    #os.environ["OPENAI_API_KEY"] = chave_api

    embeddings = OpenAIEmbeddings(
        model="text-embedding-3-small",
        openai_api_key = chave_api
    )

    # Adiciona os documentos ao Qdrant
    qdrant = Qdrant.from_documents(
        documents=documentos,
        embedding=embeddings,
        location=":memory:",  # Banco de dados temporário, armazenado em memória
        collection_name="chatbot"
    )
    return qdrant

# Carregar PDFs
qdrant = carregar_pdfs_para_qdrant("./meus_pdfs")

# Função para criar prompt personalizado usando RAG
def custom_prompt(query: str):
    # Busca documentos similares à query no Qdrant
    results = qdrant.similarity_search(query, k=3)
    source_knowledge = "\n".join([x.page_content for x in results])

    # Prompt aumentado com contexto
    augment_prompt = f"""Use o contexto abaixo para responder à pergunta.

    Contexto:
    {source_knowledge}

    Pergunta: {query}"""

    return augment_prompt

# Limite de histórico de mensagens
def limitar_mensagens(messages, max_messages=7):
    if len(messages) > max_messages:
        messages = messages[-max_messages:]  # Mantém apenas as últimas mensagens
    return messages

# Testando o fluxo com uma query
query = str(input("Usuário: "))
prompt = HumanMessage(content=custom_prompt(query))

# Lista de mensagens com limite
messages = [prompt]
messages = limitar_mensagens(messages, max_messages=5)

# Faz a chamada ao modelo com as mensagens
res = chat.invoke(messages)

# Exibe a resposta do chatbot
print("RAG_FCI : ", res.content)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
import openai

chave_api = "sua_chave_aqui"
openai.api_key = chave_api

def enviar_mensagem(mensagem, lista_mensagens=[]):
    lista_mensagens.append(
        {"role": "user", "content": mensagem}
        )

    resposta = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        messages = lista_mensagens,
    )

    return resposta["choices"][0]["message"]

lista_mensagens = []
while True:
    texto = input("Escreva aqui sua mensagem:")

    if texto == "sair":
        break
    else:
        resposta = enviar_mensagem(texto, lista_mensagens)
        lista_mensagens.append(resposta)
        print("Chatbot:", resposta["content"])
# print(enviar_mensagem("Em que ano Eistein publicou a teoria geral da relatividade?"))

utilizando como métrica de similaridade de vetores a distância de cossenos
TESTAR COM AS DUAS MÉTRICAS

DÚVIDA:
construir os chunks a cada nova consulta, ou já deixar armazenado no vectorial database os chunks de cada documento?

Testes de validação do modelo:

- testar possiveis valores de k e sua eficiência no modelo (similaridade)

- fazer a matriz de confusão e outras métricas (?)

- nem toda pergunta exige uma consulta na base de dados (exemplo: "oi, tudo bem?")
